# Démo Colab 

- Étudiant : Aldiouma MBAYE
- Professeur : M. DERRAZ Foued
- Module : Data Warehousing et ETL

- Exécutez les cellules dans l’ordre (1 → 6).
- La cellule 1 clone/pull le dépôt GitHub.
- La cellule 2 installe `pyspark` et `requests`.
- La cellule 3 télécharge les fichiers IMDb dans `raw/`.
- La cellule 4 lance le pipeline avec `--show-counts`.
- La cellule 5/6 vérifie les sorties Parquet et affiche un aperçu.


In [ ]:
# 1) Définir l'URL du dépôt GitHub (adapter si nécessaire)
REPO_URL = "https://github.com/maldiouma/Pipeline-PySpark-ETL-IMDb.git"  # Remplacer si votre URL diffère
REPO_DIR = "/content/Pipeline-PySpark-ETL-IMDb"

import os, subprocess, sys
print('REPO_URL:', REPO_URL)
if not os.path.exists(REPO_DIR):
    !git clone $REPO_URL $REPO_DIR
else:
    print('Dépôt déjà présent, pull...')
    %cd $REPO_DIR
    !git pull
%cd $REPO_DIR
print('Répertoire courant:', os.getcwd())

In [ ]:
# 2) Installer les dépendances nécessaires (pyspark, requests)
!pip install -q -r requirements.txt || pip install -q pyspark requests

In [ ]:
# 3) Préparer les dossiers et télécharger les fichiers IMDb
import os
os.makedirs('raw', exist_ok=True)

!wget -O raw/title.basics.tsv.gz https://datasets.imdbws.com/title.basics.tsv.gz
!wget -O raw/title.ratings.tsv.gz https://datasets.imdbws.com/title.ratings.tsv.gz

In [ ]:
# 4) Exécuter le pipeline avec affichage des volumes
!python src/etl_imdb.py --raw-dir raw --dw-dir dw --marts-dir marts --show-counts

In [ ]:
# 5) Vérifications des sorties (DW + Marts)
import glob, pandas as pd
print('DW:', glob.glob('dw/*'))
print('Marts:', glob.glob('marts/*'))

# Aperçu du mart year KPI
df_kpi = pd.read_parquet('marts/mart_year_kpi')
df_kpi.sort_values('yearkey').head(10)

In [ ]:
# 6) (Optionnel) Aperçu Top 10 par genre et année
import pandas as pd
df_top = pd.read_parquet('marts/mart_top_genre_year')
df_top.sort_values(['yearkey','genrekey','rk']).head(20)

In [ ]:
# 7) Aperçu distribution des notes par année
import pandas as pd
df_dist = pd.read_parquet('marts/mart_rating_distribution')
df_dist.head(20)

In [ ]:
# 8) Aperçu TOP par année (avg_rating, seuil min_votes)
import pandas as pd
df_top_year = pd.read_parquet('marts/mart_top_year_by_rating')
df_top_year.sort_values(['yearkey','rk']).head(20)